## To-do list
- metrics functions
- check for directories structure (model for prediction, databases, saving directory for the drawdown/VaR/etc plots, backup directory for model, backup directory for plots)
- import metrics (VaR, cVaR, drawdown, etc.) functions from metrics module
- functions module maybe?
- Binance access and etc (copy from Touring)
- Copy keys.py explanation from Touring
- Two main fund strategies: (1) passive - trade on 1st of each month, keep in basked the 10 crytpos with the most market cap; (2) active - trade every week based on predicted returns for the next week
- Let the user define how many assets to invest into
- Tem que usar sentiment analysis de twitter, reddit, google news, etc. Trabalhar com dados em T+0 e em T-1, T-2, T-3...
- Faz defasagem para RSI também
- Ver se tem algum dado de taxa de juros americana/europeia/chinesa(?)/japonesa diária
- Ou algum dado 'mundial' para ajudar na intepretação dos dados
- Também trabalhar com defasagens (1, 2, 3...) nesses dados
- When using Markovwitz, use pypfopt package


In [2]:
#------ Import basic packages
#import matplotlib.pyplot as plt
#import seaborn as sns
#import smtplib  # Needed for the e-mail reports
#import binance.enums  # Responsible for trading

#import numpy as np
import pandas as pd
#import pandas_ta
from functions import *
pd.set_option('display.float_format', lambda x: '%.8f' % x)

In [4]:
#from metrics import mvcriterion, optim_mvcrit

#wallet = binance_wallet()

# fetch_data() checks for the most liquid cryptos and fetches the data from Binance
data = fetch_data(days=365*2, interval='1d')
#data.drop('cluster_num', axis=1, inplace=True)

# Let's calculate the technical indicators
data = estimate_indicators(data, rsi=20, bbands=20, roll=13)

# Now we need to estimate the optimal number of clusters to
# allocate our resources
clust = estimate_kmeans(data.drop('close', axis=1), cutoff=0.25)

# Sets the params for the clustering function used with apply below
params2 = {'clusters':clust,
           'init':1}

# Groups the data and applies the clustering
data = data.dropna().groupby('time', group_keys=False).apply(clustering, **params2)

Checking for trading pairs...
The resources directory exists, checking for trade pairs file.
Trading pairs file 'thiago.old' found, checking version.


IndexError: list index out of range

In [ ]:
import os
resources_dir = './resources/'
backup_dir = 'older_versions/'

    # Check for any pair file. If it exists and is newer than a month, loads it.
arqs = os.listdir(resources_dir)

    # Keeps only generated files, disregarding folders or other misc files
for i in arqs:
    if 'pairs' not in str(i):
        print(f'ESSA MERDA IMPRIME MAS NÃO REMOVE PELO JEITO, i = {i}')
        arqs.remove(i)

#print(arqs)
"""
if len(arqs) == 0:
    print('Trading file not found in folder. Creating would be here...')
    

elif len(arqs) == 1:
    arqs = arqs[0]
    print(f"Trading pairs file '{arqs}' found, checking version.")
    file_found = arqs.split('.csv')[0]
    file_found = file_found.split('_')[1]
"""

ESSA MERDA IMPRIME MAS NÃO REMOVE PELO JEITO, i = older_versions
['thiago.old']


'\nif len(arqs) == 0:\n    print(\'Trading file not found in folder. Creating would be here...\')\n\n\nelif len(arqs) == 1:\n    arqs = arqs[0]\n    print(f"Trading pairs file \'{arqs}\' found, checking version.")\n    file_found = arqs.split(\'.csv\')[0]\n    file_found = file_found.split(\'_\')[1]\n'

In [13]:
for name in arqs:
    if 'pairs' not in str(name):
        print(f'ESSA MERDA IMPRIME MAS NÃO REMOVE PELO JEITO, i = {name}')
        arqs.remove(name)

ESSA MERDA IMPRIME MAS NÃO REMOVE PELO JEITO, i = older_versions


In [82]:
arqs = os.listdir(resources_dir)

print(arqs)

for i in arqs:
    if 'pairs' in i:
        print(f'File {i} found!')
    else:
        print(f'{i} is not a crucial file')
        temp = pd.Series(arqs) == i
        num = temp[temp.index == True].index[0]
        pop = int(num)
        arqs.pop(pop)


len(arqs)

['thiago (Copy 4).old', 'older_versions', 'thiago.old', 'thiago (Copy 3).old', 'thiago (Copy 2).old', 'thiago (Copy 5).old', 'thiago (Copy).old']
thiago (Copy 4).old is not a crucial file
thiago.old is not a crucial file
thiago (Copy 2).old is not a crucial file
thiago (Copy).old is not a crucial file


3

In [238]:
data.drop('cluster_num', axis=1, inplace=True)

In [ ]:
data.drop('USDCUSDT', axis=1, inplace=True)

close                                                 \
asset         ADAUSDT    AVAXUSDT   DOGEUSDT    LINKUSDT     LTCUSDT   
time                                                                   
2024-07-28 0.40310000 27.01000000 0.12843000 13.53000000 73.69000000   
2024-08-11 0.33900000 21.26000000 0.10791000 10.56000000 61.50000000   
2024-08-25 0.36610000 25.94000000 0.10520000 11.83000000 63.01000000   
2024-09-08 0.34350000 23.84000000 0.10376000 10.53000000 61.42000000   
2024-09-22 0.36370000 27.39000000 0.10807000 11.39000000 66.96000000   
2024-10-06 0.35120000 26.56000000 0.10841000 11.16000000 64.77000000   
2024-10-20 0.36090000 27.75000000 0.14367000 11.71000000 70.77000000   
2024-11-03 0.32630000 22.72000000 0.15822000 10.26000000 65.56000000   
2024-11-17 0.73250000 35.48000000 0.37113000 15.26000000 89.22000000   
2024-12-01 1.20220000 52.27000000 0.42457000 25.23000000         NaN   
2024-12-15 1.07750000 48.98000000 0.40195000 28.73000000         NaN   
2024-12-29 0.86150000 35.97000000 0.31396000 20.58000000         NaN   
2025-01-12 0.94440000 35.29000000 0.33807000 19.35000000         NaN   
2025-01-26 0.93450000 34.20000000 0.33397000 24.21000000         NaN   
2025-02-09 0.71210000 25.67000000 0.25542000 18.81000000         NaN   
2025-02-23 0.68160000 21.88000000 0.21043000 15.26000000         NaN   
2025-03-09 0.67120000 16.26000000 0.15408000 12.65000000         NaN   
2025-03-23 0.73200000 21.48000000 0.18300000 15.13000000         NaN   
2025-04-06 0.58640000 16.68000000 0.14958000 11.45000000         NaN   
2025-04-20 0.62420000 19.92000000 0.15939000 13.11000000         NaN   
2025-05-04 0.66220000 19.69000000 0.17062000 13.65000000         NaN   
2025-05-18 0.74250000 22.25000000 0.22445000 15.78000000         NaN   
2025-06-01 0.68960000 21.15000000 0.19507000 14.07000000         NaN   
2025-06-15 0.63030000 19.06000000 0.17272000 13.56000000         NaN   
2025-06-29 0.57220000 17.96000000 0.16518000 13.38000000         NaN   
2025-07-13 0.73520000 21.26000000 0.19749000 15.75000000         NaN   
2025-07-27 0.80290000 23.48000000 0.23889000 17.74000000         NaN   

                                                                   ...  \
asset        SHIBUSDT    SUIUSDT    TRXUSDT   USDCUSDT    XRPUSDT  ...   
time                                                               ...   
2024-07-28 0.00001662 0.74000000 0.13750000 1.00000000 0.60190000  ...   
2024-08-11 0.00001391 1.01430000 0.12670000 0.99970000 0.56840000  ...   
2024-08-25 0.00001461 0.92650000 0.16160000 0.99970000 0.58740000  ...   
2024-09-08 0.00001354 0.95810000 0.15460000 0.99970000 0.53950000  ...   
2024-09-22 0.00001448 1.53480000 0.15230000 1.00000000 0.58470000  ...   
2024-10-06 0.00001743 2.01730000 0.15590000 1.00020000 0.52860000  ...   
2024-10-20 0.00001836 2.00960000 0.15840000 1.00040000 0.54490000  ...   
2024-11-03 0.00001686 1.84950000 0.16290000 1.00060000 0.50350000  ...   
2024-11-17 0.00002486 3.69920000 0.20220000 0.99930000 1.11370000  ...   
2024-12-01 0.00002992 3.38940000 0.22100000 0.99970000 2.72450000  ...   
2024-12-15 0.00002719 4.62860000 0.29670000 0.99970000 2.48410000  ...   
2024-12-29 0.00002114 4.18110000 0.25340000 1.00180000 2.05870000  ...   
2025-01-12 0.00002120 4.74870000 0.22280000 1.00030000 2.52320000  ...   
2025-01-26 0.00001897 3.96360000 0.24650000 1.00010000 3.05520000  ...   
2025-02-09 0.00001580 3.21590000 0.24620000 0.99990000 2.42370000  ...   
2025-02-23 0.00001364 2.83040000 0.24010000 1.00010000 2.27830000  ...   
2025-03-09 0.00001150 2.08880000 0.22920000 1.00040000 2.02160000  ...   
2025-03-23 0.00001353 2.39750000 0.22820000 0.99990000 2.45010000  ...   
2025-04-06 0.00001142 2.01910000 0.22870000 1.00040000 1.89870000  ...   
2025-04-20 0.00001236 2.19640000 0.24630000 1.00000000 2.08550000  ...   
2025-05-04 0.00001269 3.42340000 0.24830000 1.00020000 2.13060000  ...   
2025-05-18 0.00001460 3.83730000 0.26670000 0.99960000 2.38030000  ...   
2025-

In [240]:
data[data.rsi > 55].cluster_num.value_counts()

cluster_num
0    66
1    11
3     9
5     8
2     8
4     6
Name: count, dtype: int64

In [ ]:
for i in data.index.get_level_values('time').unique().to_list():
    g = data.xs(i, level=0)
    plot_clusters(g, attr_1='close_norm')

### Down here is the tweaked functions:

In [3]:
#------ Indicators function
def estimate_indicators(df, rsi=20, bbands=20, roll=13):
    """
    Description: gets the data from the web and calculates its indicators

    Input:  rsi (int), time window for the RSI indicator. Defaults to 20;
            bbands (int), time window for the Bollinger Bands indicator. Defaults to 20;
            roll (int), rolling window for moving average. Defaults to 13.
    """
    import pandas as pd
    import pandas_ta
    import numpy as np

    # Close prices standardization
    print('Standardizing close prices')
    df['close_norm'] = df['close'].groupby('asset').transform(lambda x: (x - x.mean())/x.std())

    print('Calculating RSI (momentum).')
    # Calculating Relative Strenght Index (RSI) - momentum indicator
    # The RSI indicator won't be standardized for its use in clustering
    df['rsi'] = df.groupby(level=0)['close'].transform(lambda x: pandas_ta.rsi(close=x, length=rsi))
    #df.xs('BTCUSDT', level=0)['rsi'].plot() # to check if it's worked just uncomment the beginning of this line

    print('Calculating Bollinger Bands (vol).')
    # Calculating Bollinger Bands - volatility indicator (overbought/oversold)
    df['bb_low'] = df.groupby(level=0)['close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=bbands).iloc[:,0])
    df['bb_mid'] = df.groupby(level=0)['close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=bbands).iloc[:,1])
    df['bb_high'] = df.groupby(level=0)['close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=bbands).iloc[:,2])

    print('Calculating ATR (vol).')
    # Average True Range (ATR) - volatility indicator
    # Since this function uses 3 features to compute the indicator (high, low, close), it is needed to use 'apply' instead of 'transform',
    # and for that a custom function is needed (check it in Functions section).
    df['atr'] = df.groupby(level=0, group_keys=False).apply(atr_calc)

    print('Calculating MACD (momentum).')
    # Moving Average Convergence-Divergence (MACD) - momentum indicator
    # Same reasonig as ATR, a custom function is needed here.
    df['macd'] = df.groupby(level=0, group_keys=False).apply(macd_calc).iloc[:,0]

    print('Calculating dollar volume in millions.')
    # Dollar volume (based on closing price), divided by 1mil
    df['dollar_vol'] = df['volume']*df['close']/1e6

    print('Aggregating data to bi-weekly periods, filtering best cryptos.')
    # Aggregate to bi-weekly level and filter N most market capped cryptos
    indicators = [c for c in df.columns.unique() if c not in ['dollar_vol', 'open', 'high', 'low', 'volume']]
    p_dvol = df.unstack(level=0)['dollar_vol'].resample('2W').mean().stack('asset').to_frame('dollar_vol')
    p_indc = df.unstack(level=0)[indicators].resample('2W').last().stack('asset', future_stack=True)
    df = pd.concat([p_dvol, p_indc], axis=1).dropna()

    print('Creating dollar volume moving averages.')
    # 13-week moving average of dollar volume for each asset
    df['dollar_vol'] = df['dollar_vol'].unstack('asset').rolling(roll).mean().stack()

    print('Checking cryptos liquidity.')
    # Bi-weekly rank for each asset by dollar volume (a.k.a. liquidity), smaller rank is better (most liquid)
    df['liquidity_lvl'] = df.groupby('time')['dollar_vol'].rank(ascending=False)

    print('Creating a rank for the best cryptos in the dataset.')
    # Top 15 cryptos fortnightly, able to drop volume and liquidity features already
    mask = df['liquidity_lvl'] < 16
    df = df.loc[mask].drop(['dollar_vol', 'liquidity_lvl'], axis=1)

    print('Estimating returns.')
    df = df.groupby(level='asset', group_keys=False).apply(estimate_returns).dropna()

    print('All indicators successfuly calculated.')

    return df


#------ Function to estimate the optimal k value for clustering
def estimate_kmeans(data, max_k=10, cutoff=0.125, graph=False):
    """
    Description: estimates the optimal k value for clustering based on a cutoff value

    Input: days (pandas Series or DataFrame), the values to perform the clustering analysis;
            max_k (int), the maximum number of clusters. Defaults to 10;
            cutoff (float), the percentage (in decimals) in change to evaluate. Defaults to 0.125;
            graph (bool), whether plot a graph or not. Defaults to False

    Output: opt_k (int), the theorical optimum value of k
    """
    from sklearn.cluster import KMeans
    import pandas as pd
    
    # Creates empty lists to store mean and intertia values
    means = []
    inertias = []

    # Cycle through data to calculate means and inertias 
    for k in range(1, max_k):
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(pd.DataFrame(data))

        means.append(k)
        inertias.append(kmeans.inertia_)

    # Creates a pandas DataFrame with the results
    calc = pd.DataFrame([means, inertias], index=['Means', 'Inertia']).T

    # Checks which inertias are under the cutoff value and defines the best k under this assumption
    try:
        mask = (calc.Inertia / calc.Inertia[0]) < cutoff
        opt_k = int(calc[mask].Means.min())
    
        # Plot the elbow graph if graph param is set to True
        if graph == True:
            import matplotlib.pyplot as plt

            if cutoff != 0.125:
                print(f'Theoretical best k: {opt_k}, with change cutoff {cutoff}')
            else:
                print(f'Theoretical best k: {opt_k}, with default change cutoff value {cutoff}')

            fig = plt.subplots(figsize=(10,5))
            plt.plot(means, inertias, 'o-')
            plt.xlabel('Number of Clusters (k)')
            plt.ylabel('Inertia')
            plt.grid(True)
            plt.show()
            print(f'WARNING! Clustering not registered in the dataset. For it to be done, use hyperparameter "graph=False".')

        else:
            return opt_k

    except ValueError as e:
        return print(f'ERROR! Hyperparameter "cutoff" too low, try to use a greater value')

    

#------ Clustering itself
def clustering(df, metric='rsi', clusters=None, init=0):
    """
    Description: performs the clustering of the data, based on estimate_kmeans() results.

    Input: df (pandas DataFrame), the data to perform the clustering;
            metric (str), metric to estimate the centroids. Defaults to 'rsi';
            clusters (int), either user-defined or the results from estimate_kmeans();
            init (int), 0 (k-means++), 1 (centroids) or 2 (random). Defaults to 0.

    Output: df (pandas DataFrame), the clustered DataFrame
    """
    import pandas as pd
    from sklearn.cluster import KMeans
    import numpy as np
    
    temp = pd.Series(df.columns == metric)
    num = temp[temp.index == True].index[0]
    feat = int(num)

    # Set target values
    target_values = []
    for num in range(1, clusters+1):
        temp = int(num * np.quantile(range(0, 101), 1/(clusters+1)))
        target_values.append(temp)    

    # Calculate centroids
    centroids = np.zeros((len(target_values), len(df.columns)))
    centroids[:, feat] = target_values
    
    if init == 0:
        df['cluster_num'] = KMeans(n_clusters=clusters,
                                   random_state=1,
                                   init='k-means++').fit(df).labels_
        return df
    
    elif init == 1:
        df['cluster_num'] = KMeans(n_clusters=clusters,
                                   random_state=1,
                                   init=centroids).fit(df).labels_
        return df

    elif init == 2:
        df['cluster_num'] = KMeans(n_clusters=clusters,
                                   random_state=1,
                                   init='random').fit(df).labels_
        return df
    
    else:
        print('"init" argument MUST be 0 (k-means++), 1 (centroids) or 2 (random)')
    